## CSV-Dateien der RCPs zusammenfügen

In [4]:
import pandas as pd

# Einlesen und Spalten umbennen
rcp26_tas = pd.read_csv("https://raw.githubusercontent.com/emorl/projektstudie-murgtal/main/1_daten/rcp26pr/murg_pr_rcp26_mean.csv").rename(columns={"precip_mm_day": "pr_day_rcp26"})
rcp85_tas = pd.read_csv("https://raw.githubusercontent.com/emorl/projektstudie-murgtal/main/1_daten/rcp26temp/murg_tas_rcp26_mean.csv").rename(columns={"temp_day": "tas_day_rcp26"})
rcp26_pr   = pd.read_csv("https://raw.githubusercontent.com/emorl/projektstudie-murgtal/main/1_daten/rcp85pr/murg_pr_rcp85_mean.csv").rename(columns={"precip_mm_day": "pr_day_rcp85"})
rcp85_pr   = pd.read_csv("https://raw.githubusercontent.com/emorl/projektstudie-murgtal/main/1_daten/rcp85pr/murg_pr_rcp85_mean.csv").rename(columns={"temp_day": "tas_day_rcp85"})

# date als Datetime-Objekt
for df in [rcp26_tas, rcp85_tas, rcp26_pr, rcp85_pr]:
    df["date"] = pd.to_datetime(df["date"])

# Zusammenführen über date-Spalte
merged_df = rcp26_tas.merge(rcp85_tas, on="date") \
                      .merge(rcp26_pr, on="date") \
                      .merge(rcp85_pr, on="date")

#lokal speichern
output_csv = "hier-lokalen-Pfad-definieren/1_daten/tas_pr_merge.csv" 
merged_df.to_csv(output_csv, index=False)

print(merged_df.head())


                 date  pr_day_rcp26  tas_day_rcp26  pr_day_rcp85  \
0 2006-01-01 12:00:00      0.116667       3.933350      0.000000   
1 2006-01-02 12:00:00      0.000000       2.583313      0.016667   
2 2006-01-03 12:00:00      0.083333       1.049988      1.100000   
3 2006-01-04 12:00:00      2.416667       2.716675      0.333333   
4 2006-01-05 12:00:00     31.266668     -13.016663      0.633333   

   precip_mm_day  
0       0.000000  
1       0.016667  
2       1.100000  
3       0.333333  
4       0.633333  


# Abflussdaten vorbereiten und neu speichern

In [5]:
import pandas as pd

# CSV ab Zeile 7 einlesen mit korrekter Kodierung, da erste 6 Zeilen Projektbeschreibung sind
df = pd.read_csv(
    "https://raw.githubusercontent.com/emorl/projektstudie-murgtal/main/1_daten/discharge/Hydrologische-Landespegel-2025-12-22.csv",
    encoding="ISO-8859-1",  
    sep=",",                
    skiprows=6,             
    decimal=",",            
    quotechar='"'          
)

# Anfang anzeigen lassen
print(df.head())

   Messstellennummer        Stationsname Gewässer Parameter Datum / Uhrzeit  \
0               1301  Bad Rotenfels Murg     Murg         Q      1981-01-01   
1               1301  Bad Rotenfels Murg     Murg         Q      1981-01-02   
2               1301  Bad Rotenfels Murg     Murg         Q      1981-01-03   
3               1301  Bad Rotenfels Murg     Murg         Q      1981-01-04   
4               1301  Bad Rotenfels Murg     Murg         Q      1981-01-05   

  Zeitbezug    Wert Einheit          Produkt  
0       MEZ  17.644    m³/s  Tagesmittelwert  
1       MEZ  23.305    m³/s  Tagesmittelwert  
2       MEZ  53.963    m³/s  Tagesmittelwert  
3       MEZ  75.475    m³/s  Tagesmittelwert  
4       MEZ  42.074    m³/s  Tagesmittelwert  


In [6]:
# Datum umwandeln
df["Datum / Uhrzeit"] = pd.to_datetime(df["Datum / Uhrzeit"], errors="coerce")

# Nur Datum und Abflusswerte behalten
df = df[["Datum / Uhrzeit", "Wert"]].rename(columns={
    "Datum / Uhrzeit": "date",
    "Wert": "flow_m3_s"
})

print(df.head())


        date  flow_m3_s
0 1981-01-01     17.644
1 1981-01-02     23.305
2 1981-01-03     53.963
3 1981-01-04     75.475
4 1981-01-05     42.074


In [7]:
#Anzeigen lassen
df.describe()

,date,flow_m3_s
count,16071,16071.000000
mean,2002-12-31 23:59:59.999999872,15.351894
min,1981-01-01 00:00:00,1.428000
25%,1992-01-01 12:00:00,5.962000
50%,2003-01-01 00:00:00,9.771000
75%,2013-12-31 12:00:00,17.750500
max,2024-12-31 00:00:00,385.156000
std,NaN,17.873557


In [13]:
# Neue Datei speichern
out_csv = "hier-lokalen-Pfad-definieren/1_daten/discharge/discharge_clean.csv"
df.to_csv(out_csv, index=False, encoding="utf-8")
